### Test script for multiturn conversation dataset generation

* Cell 1: load packages
* Cell 2: Generate 100000 system prompts to check for potential bugs in the code
* Cell 3 and further: Generate a conversation example

In [1]:
import json
from tqdm import tqdm
from openai import OpenAI
import random
import pandas as pd
from multiturn_modul import MultiturnStyle, get_multiturn_style

path_to_api_key: str = "../API_KEY.txt"
my_api_key = open(path_to_api_key, 'r').read()
client = OpenAI(api_key=my_api_key)
gpt_model: str = "gpt-4o"

In [2]:
countries = pd.read_csv("../resources/countries_by_income_category.csv")
sampled_country = random.choice(countries.iloc[:, 0].tolist())
id = "0-122"

for i in range(100000):
    test = get_multiturn_style(str(i%12)+"-112", "Switzerland")
    test.system_prompt_chatbot()
    test.system_prompt_user()

In [3]:
# Test mode
# Generate multiturn conversations between Meditron (simulated by gpt-4o) and a Physician (simulated by gpt-4o)

# System prompts
system_prompt_chatbot = test.system_prompt_chatbot()
system_prompt_user = test.system_prompt_user()  

# store medical AI tasks in a list
path_to_prompts = "../results/parsed_prompts_tasks_x_subtopics.json" 
with open(path_to_prompts, 'r') as file:
    task_x_subtopics_prompts = json.load(file)  # Load JSON data from file
initial_physician_prompt = task_x_subtopics_prompts[100]["prompt"]
print(initial_physician_prompt)

# Function that generates the next step of a conversation

def next_conversation_step(system_prompt, chat_histroy, verbose = False):
    if system_prompt == "user":
        content = system_prompt_user
    else:
        content = system_prompt_chatbot

    prompt = chat_histroy

    if verbose:
        print("Send prompt to GPT:")
        print("#########")
        print(prompt)
        print("########")

    completion = client.chat.completions.create(
        model= gpt_model,
        messages=[
            {"role": "system", "content": content},
            {
                "role": "user",
                "content": f"{prompt}"
            }
        ]
    )
    response = completion.choices[0].message.content
    
    if verbose:
        print("Receiving responses from GPT...")
        print(response)
    return chat_histroy + "\n" + f"({system_prompt}) " + response

A 32-year-old male patient presents with a 3-day history of high fever, chills, and body aches. He also reports a sore throat and a non-productive cough. He returned from a business trip to Southeast Asia 10 days ago. His medical history is notable for mild asthma. On examination, he has a temperature of 102°F, mild pharyngeal erythema, and bilateral cervical lymphadenopathy. Considering his recent travel history, could this presentation be indicative of an emerging infectious disease endemic to that region, and what diagnostic tests should be prioritized?


In [ ]:
# Check if everything is fine
N = test.number_of_turns
print(N)
print(f"chatbot: {system_prompt_chatbot}")
print(f"user: {system_prompt_user}")

3
chatbot: You are a medical AI chatbot designed to assist health care workers working in Switzerland by answering their questions. 
The style of your answers must follow these rules: 
- Respectful, polite interaction: You must always engage in a respectful, polite, and courteous manner, maintaining professionalism in all interactions.
- Honest, evidence-based information: All responses have to be based on the latest medical evidence and guidelines
- Ethical and safe content: Under no circumstances should you provide fake, harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. If a question is unclear or factually incorrect, you should explain why rather than attempting to answer it inaccurately.
- Dosage: If asked about dosage medication, provide a ballpark estimate of the dosage but always provide a concise warning that this should be checked in the form of “(dosages should be verified)”.
- Tailor responses to the geographical context, resource setting, level of ca

In [ ]:
conversation_history = []
conversation_history.append("(user) " + initial_physician_prompt)
for i in ### tqdm(range(2*N - 1)): ### Change range -> will cost money
    if i % 2 == 0:
        conversation_history.append(next_conversation_step("chatbot", conversation_history[-1]))
    else:
        conversation_history.append(next_conversation_step("user", conversation_history[-1]))


100%|██████████| 5/5 [00:16<00:00,  3.21s/it]


In [6]:
print(conversation_history[-1])

(user) A 32-year-old male patient presents with a 3-day history of high fever, chills, and body aches. He also reports a sore throat and a non-productive cough. He returned from a business trip to Southeast Asia 10 days ago. His medical history is notable for mild asthma. On examination, he has a temperature of 102°F, mild pharyngeal erythema, and bilateral cervical lymphadenopathy. Considering his recent travel history, could this presentation be indicative of an emerging infectious disease endemic to that region, and what diagnostic tests should be prioritized?
(chatbot) Given the patient's recent travel to Southeast Asia and the symptoms presented, it would be prudent to consider several infectious diseases that are endemic in that region. Dengue fever, chikungunya, and influenza are common infectious diseases that should be considered.

Additionally, emerging diseases such as Zika virus or even other viral respiratory infections, like COVID-19, may also be relevant. 

For diagnosti